In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = read_csv('https://raw.githubusercontent.com/datagy/mediumdata/master/AAPL.csv', parse_dates=['Date'])

print(df.head())

NameError: name 'read_csv' is not defined

In [11]:
import pandas as pd

url_data = (r'https://raw.githubusercontent.com//tvelichkovt/PyTorch/main/flights.csv')

data_csv = pd.read_csv(url_data)

data_csv.head()

HTTPError: HTTP Error 400: Bad Request